In [220]:
import numpy as np
from numpy import array
import pickle as plk
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer

In [4]:
df = pd.read_excel('/Users/luyin/Desktop/project/Q&A.xlsx',header = 0)
l = df['Breakout'].unique() # 79 unique analyst
dic = {} #create dictionary for questions
for category in l:
    list_ = list(df.loc[df['Breakout']  == category]['Question'])
    dic[category] = list_

In [126]:
import spacy
import re
import string
# Load English tokenizer, tagger, parser, NER and word vectors
UNK_IDX = 0


#tokenize sentence by sentence
def question_split(input_):
    list_ = []
    for q in input_:
        q = q.split('\n')
        if len(q) ==1:
            list_.append(q)
    return list_
    
def tokenize(sent):
    sent = re.sub('y ou', 'you', sent)
    sent = re.sub('y es', 'yes', sent)
    sent = re.sub('v o', 'vo', sent)
    sent = re.sub("don't", 'dont', sent)
    sent = re.sub('[^A-Za-z&]', ' ', sent) # replace non-letter with space
#     sent = re.sub(r'\b[a-zA-Z]\b', '', sent) #
#     sent = re.sub('^[0-9]+', '', sent)
    return sent.split()

# tokens = tokenize(" ? y ou FICC going dont v ote to hit them one way or another strong dollar did seem to have a huge impact and y ou, what are you doing? I'm 's what do you and me think or like apples and Apple is looking at buying and bought U.K. startup for $1 billion. '\n' another sentence")
# print(tokens)

['you', 'FICC', 'going', 'dont', 'vote', 'to', 'hit', 'them', 'one', 'way', 'or', 'another', 'strong', 'dollar', 'did', 'seem', 'to', 'have', 'a', 'huge', 'impact', 'and', 'you', 'what', 'are', 'you', 'doing', 'I', 'm', 's', 'what', 'do', 'you', 'and', 'me', 'think', 'or', 'like', 'apples', 'and', 'Apple', 'is', 'looking', 'at', 'buying', 'and', 'bought', 'U', 'K', 'startup', 'for', 'billion', 'another', 'sentence']


In [188]:
#load training and validation dataset
dic_len = len(dic['Balance sheet'])
all_data = dic['Balance sheet']
train_data = dic['Balance sheet'][:500]
val_data = dic['Balance sheet'][500: dic_len]

In [189]:
all_tokens = []
for q in dic['Balance sheet']:
    tokens = tokenize(q)
    all_tokens += tokens

In [190]:
len(all_tokens)

52578

In [191]:
SEQUENCE_LENGTH = 20 
# organize into sequences of tokens
def build_sequence(length, input_):
    sequences = list()
    for l in range(length, len(input_)):
        seq = input_[l-length:l]
        line = ' '.join(seq)
        sequences.append(line)
#     print('Total Sequences: {}' .format(len(sequences)))
    return sequences

In [196]:
def data_generator(dic, length):
    out_list = []
    raw = question_split(dic)
    for i in range(len(raw)):
        sent = tokenize(raw[i][0].lower())
        q_sequence = build_sequence(length, sent)
        out_list += q_sequence
    return out_list

In [197]:
all_d = data_generator(all_data,20)
train_d = data_generator(train_data,20)
val_d = data_generator(val_data,20)

In [198]:
val_d = data_generator(val_data,20)

In [203]:
all_d[:5]

['on net interest income do you have an outlook for how the net interest income dollars could trend from here',
 'net interest income do you have an outlook for how the net interest income dollars could trend from here assuming',
 'interest income do you have an outlook for how the net interest income dollars could trend from here assuming that',
 'income do you have an outlook for how the net interest income dollars could trend from here assuming that you',
 'do you have an outlook for how the net interest income dollars could trend from here assuming that you don']

In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(all_tokens) #fit on texts
# sequences = tokenizer.texts_to_sequences(data_input)
# vocab_size = len(tokenizer.word_index) + 1

In [147]:
def data_loader(data_input):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(all_tokens) #fit on texts
    sequences = tokenizer.texts_to_sequences(data_input)
    vocab_size = len(tokenizer.word_index) + 1
    X = []
    y = []
    for i in range(len(sequences)):
        X.append((sequences[i][:(SEQUENCE_LENGTH-1)]))
        y.append((sequences[i][-1]))

    y = array(y)
    X = array(X)
    y = to_categorical(y, num_classes=vocab_size)
    return X, y

In [212]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tokens)

In [148]:
train_input, train_label =  data_loader(train_d)

In [149]:
val_input, val_label =  data_loader(val_d)

In [152]:
print(len(train_input), len(train_label), len(train_label[0]))
print(len(val_input), len(val_label), len(val_label[0]))

16037 16037 3078
6028 6028 3078


In [183]:
train_input[0]

array([ 10,  90,  60, 121,  36,   2,  35,  84, 148,  22,  23,   1,  90,
        60, 121, 679,  49, 479,  42])

In [208]:
def save_file(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

out_filename = 'Balance sheet.txt'
save_file(all_d, out_filename)
# load doc into memory
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
in_filename = 'Balance sheet.txt'
doc = load_file(in_filename)
lines = doc.split('\n')

In [210]:
from random import randint
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):
    # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
    # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [158]:
# define model
model = Sequential()
model.add(Embedding(3078, 100, input_length= SEQUENCE_LENGTH-1))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(3078, activation='softmax'))

#import the checkpoint to save current model
filepath="LSTM_new1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks_list = [checkpoint, earlystopper]
# compile model

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the model
print(len(train_input), len(train_label), len(train_label[0]))
print(len(val_input), len(val_label), len(val_label[0]))
model.fit(train_input, train_label, validation_data =(val_input, val_label), batch_size= 200, epochs=100, callbacks=callbacks_list)

# categorical_crossentropy
 
# save the model to filey
model.save('model_new1.h5')
# save the tokenizer
plk.dump(tokenizer, open('tokenizer_new.pkl', 'wb'))

16037 16037 3078
6028 6028 3078
Train on 16037 samples, validate on 6028 samples
Epoch 1/100
16037/16037 [==============================] - 27s 2ms/step - loss: 6.5261 - acc: 0.0393 - val_loss: 6.0322 - val_acc: 0.0509

Epoch 00001: loss improved from inf to 6.52612, saving model to LSTM_new1.hdf5
Epoch 2/100
16037/16037 [==============================] - 21s 1ms/step - loss: 5.8869 - acc: 0.0512 - val_loss: 6.0525 - val_acc: 0.0509

Epoch 00002: loss improved from 6.52612 to 5.88693, saving model to LSTM_new1.hdf5
Epoch 3/100
16037/16037 [==============================] - 21s 1ms/step - loss: 5.8662 - acc: 0.0512 - val_loss: 6.0789 - val_acc: 0.0509

Epoch 00003: loss improved from 5.88693 to 5.86618, saving model to LSTM_new1.hdf5
Epoch 4/100
16037/16037 [==============================] - 21s 1ms/step - loss: 5.8542 - acc: 0.0512 - val_loss: 6.0797 - val_acc: 0.0509

Epoch 00004: loss improved from 5.86618 to 5.85423, saving model to LSTM_new1.hdf5
Epoch 5/100
16037/16037 [==========

16037/16037 [==============================] - 23s 1ms/step - loss: 4.2842 - acc: 0.1990 - val_loss: 5.9657 - val_acc: 0.1466

Epoch 00037: loss improved from 4.31307 to 4.28415, saving model to LSTM_new1.hdf5
Epoch 38/100
16037/16037 [==============================] - 22s 1ms/step - loss: 4.2531 - acc: 0.2019 - val_loss: 5.9779 - val_acc: 0.1462

Epoch 00038: loss improved from 4.28415 to 4.25306, saving model to LSTM_new1.hdf5
Epoch 39/100
16037/16037 [==============================] - 22s 1ms/step - loss: 4.2223 - acc: 0.2038 - val_loss: 5.9880 - val_acc: 0.1471

Epoch 00039: loss improved from 4.25306 to 4.22233, saving model to LSTM_new1.hdf5
Epoch 40/100
16037/16037 [==============================] - 23s 1ms/step - loss: 4.1986 - acc: 0.2052 - val_loss: 6.0106 - val_acc: 0.1452

Epoch 00040: loss improved from 4.22233 to 4.19861, saving model to LSTM_new1.hdf5
Epoch 41/100
16037/16037 [==============================] - 23s 1ms/step - loss: 4.1686 - acc: 0.2064 - val_loss: 6.0139 

16037/16037 [==============================] - 22s 1ms/step - loss: 3.3419 - acc: 0.2847 - val_loss: 6.5196 - val_acc: 0.1516

Epoch 00074: loss improved from 3.35841 to 3.34186, saving model to LSTM_new1.hdf5
Epoch 75/100
16037/16037 [==============================] - 22s 1ms/step - loss: 3.3194 - acc: 0.2852 - val_loss: 6.5249 - val_acc: 0.1521

Epoch 00075: loss improved from 3.34186 to 3.31944, saving model to LSTM_new1.hdf5
Epoch 76/100
16037/16037 [==============================] - 22s 1ms/step - loss: 3.2906 - acc: 0.2921 - val_loss: 6.5483 - val_acc: 0.1493

Epoch 00076: loss improved from 3.31944 to 3.29058, saving model to LSTM_new1.hdf5
Epoch 77/100
16037/16037 [==============================] - 22s 1ms/step - loss: 3.2683 - acc: 0.2921 - val_loss: 6.5676 - val_acc: 0.1491

Epoch 00077: loss improved from 3.29058 to 3.26828, saving model to LSTM_new1.hdf5
Epoch 78/100
16037/16037 [==============================] - 22s 1ms/step - loss: 3.2478 - acc: 0.2976 - val_loss: 6.5819 

TypeError: __init__() got an unexpected keyword argument 'encoding'

In [221]:
filepath="LSTM_new1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks_list = [checkpoint, earlystopper]

model.load_weights(filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_input, train_label, validation_data =(val_input, val_label), batch_size= 200, epochs=100, callbacks=callbacks_list)


Train on 16037 samples, validate on 6028 samples
Epoch 1/100
16037/16037 [==============================] - 27s 2ms/step - loss: 2.8835 - acc: 0.3537 - val_loss: 7.0043 - val_acc: 0.1480

Epoch 00001: loss improved from inf to 2.88352, saving model to LSTM_new1.hdf5
Epoch 2/100
16037/16037 [==============================] - 22s 1ms/step - loss: 2.7484 - acc: 0.3751 - val_loss: 7.0576 - val_acc: 0.1422

Epoch 00002: loss improved from 2.88352 to 2.74839, saving model to LSTM_new1.hdf5
Epoch 3/100
16037/16037 [==============================] - 22s 1ms/step - loss: 2.7144 - acc: 0.3779 - val_loss: 7.0764 - val_acc: 0.1435

Epoch 00003: loss improved from 2.74839 to 2.71435, saving model to LSTM_new1.hdf5
Epoch 4/100
16037/16037 [==============================] - 22s 1ms/step - loss: 2.6924 - acc: 0.3831 - val_loss: 7.1094 - val_acc: 0.1447

Epoch 00004: loss improved from 2.71435 to 2.69238, saving model to LSTM_new1.hdf5
Epoch 5/100
16037/16037 [==============================] - 22s 1ms/

In [222]:
plk.dump(tokenizer, open('tokenizer_new.pkl', 'wb'))
model.save('model_new1.h5')

In [223]:
#test 1
# load cleaned text sequences
in_filename = 'Balance sheet.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('model_new1.h5')
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer_new.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

might be a one year additional grant period if needed but just remind us of how much is private that

you re putting on for the fed down how are you thinking about the industry are you seeing any signs of the industry will impact as you think that s a little bit more than construction and just isolating out that the optimal mix growth according to the balance sheet


In [216]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

touch on your deposit base which is rather unique and can you elaborate on deposit costs and beta trends outside

s basically been up i m wondering how much says how much of the balance sheet and then the consumer side and then it s due to the economy and the hedges will work and i m wondering how much more of the balance sheet and then you re thinking


In [217]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

my question i just wanted to follow up so last year it looks like you grew your average earning assets

in the commercial customers are you seeing any evidence of the fed moves how to the size of that portfolio and also the margin carry return and how you think about the fed funds sold of uncertainty in the past few years how are you thinking about the fed moves


In [218]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

two percentage points q on q and up from year on year and i m just wondering how far do

you think about the fed funds sold of uncertainty in the past few years in said it the fed funds sold of uncertainty for the fed funds rate risk and how much more banks are examples how to citigroup average balance sheet and how you re seeing a little bit
